In [1]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = 'Enter_Your_Client_Id'
CLIENT_SECRET = 'Enter_Your_Client_Secret'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"  #the client_id and client_secret are added in the client_credentials
client_credentials_base64 = base64.b64encode(client_credentials.encode())  #this client_credential is then secured using base64 encoding

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'  #post req is send to the url
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'  # Authorization holds the client credentials, as it is req for client authentication
}
data = {
    'grant_type': 'client_credentials'   # it is set to indicate that the client is requestion for an access token for the client credentials flow
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [2]:
# in the above code client_id and client_secret hold my credentials which uniquel identify the application making request
# to our spotify API
# with the help of access token now our application can make request for the authorized data about music, album artists and users

In [3]:
# We will import Spotipy library which is a python lib used to access Spotify web api
%pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)  #initialsed spotipy client to provide access token which serves as an authentication token
    # the access_token make an authorized request to access spotify resource.
    
    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')
    # the sp.plalist_tracks method retrieve the playlist tracks, the fields parameter is used to define the track information such as id, name, artists etc
    
    # Extract relevant information and store in a list of dictionaries called music_data
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None
        # this function uses sp.audio_features in order to extract the audio of each track, this audio feature include features such as dancebility,
        # enery, key, loudness etc. These feature provide the insights.

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [5]:
playlist_id = '1VuEjhXBa0GvGJf7SvrLiF'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

       Track Name                                            Artists  \
0        Bin Tere  Vishal-Shekhar, Shafqat Amanat Ali, Sunidhi Ch...   
1  Bin Tere Sanam                 Udit Narayan, Kavita Krishnamurthy   
2  Bin Tere Sanam                 Udit Narayan, Kavita Krishnamurthy   
3        Isq Risk     Sohail Sen, Rahat Fateh Ali Khan, Irshad Kamil   
4        Isq Risk     Sohail Sen, Rahat Fateh Ali Khan, Irshad Kamil   
5  Chand Se Parda                                         Kumar Sanu   
6  Chand Sifarish    Jatin-Lalit, Shaan, Kailash Kher, Prasoon Joshi   
7    Chaand Taare                                           Abhijeet   

                                          Album Name                Album ID  \
0  I Hate Luv Storys (Original Motion Picture Sou...  4SPHfSWibeoFCSdR4vd5QL   
1  Yaara Dildara (Original Mostion Picture Soundt...  1a8QGPg7O93LqKze9EvfU1   
2  Yaara Dildara (Original Mostion Picture Soundt...  1a8QGPg7O93LqKze9EvfU1   
3                             M

In [6]:
# in the above code snippet we used the playlist id as 37i9dQZF1DX76Wlfdnj7AP
# the code call get_trending_playlist_data function as defined above 
# the collected data is stored in the dataframe called music_df 

In [7]:
# checking if our data has null value or not
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [8]:
# Importing ncessary liraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [9]:
# while building a music recommendation system it is important to provide user with the recent release so uilding a fun for that

In [10]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d') # release date is taken as an input in YYYY-MM-DD
    # it used datetime.strptime from python datetime module to convert release date string to datetime object.
    
    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date  # calculate time span between release date and current date using datetime.now() for current date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [11]:
# the idea behind the formula of the weight is that the weight decrease of a music when time span increases which means it is a older song

In [12]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [13]:
# We will build a hybrid recommendation system for our this data
# A hybrid recommendation system is a recommendation technique that offers a complete and balanced approach by mixing two or more
# recommendation techniques

# We will generate music recommendations based on the music audio features

In [14]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:  # checks if music name exists in our music_df
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

# Now we will generate music recommendations based on weighted popularity and combine it with the recommendations of the content-based filtering method using the hybrid approach

In [15]:
import pandas as pd

def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]
    )

    new_entry = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, new_entry], ignore_index=True)

    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations

In [16]:
# The hybrid approach aims to provide more personalized and relevant recommendations by considering both the content similarity of songs and
# their weighted popularity

In [17]:
# Testing our model
input_song_name = "Bin Tere Sanam"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Bin Tere Sanam':
       Track Name                                            Artists  \
3  Chand Sifarish    Jatin-Lalit, Shaan, Kailash Kher, Prasoon Joshi   
2        Bin Tere  Vishal-Shekhar, Shafqat Amanat Ali, Sunidhi Ch...   
1  Chand Se Parda                                         Kumar Sanu   
4    Chaand Taare                                           Abhijeet   

                                          Album Name Release Date  Popularity  
3                                              Fanaa   2006-04-12        69.0  
2  I Hate Luv Storys (Original Motion Picture Sou...   2010-05-25        63.0  
1                                     Aao Pyar Karen   1994-11-25        58.0  
4      Yes Boss (Original Motion Picture Soundtrack)   1999-06-18        54.0  
